In [121]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
import datetime as dt

In [122]:
username = 'vigrose'
password = 'slytherin10946'

In [123]:
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [20]:
phys_query_2020 = \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    O.CUSTOMER_KEY,
    D.MONTH_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.YR = 2020
    AND
    O.ORDER_PRODUCT_ID IN (4915522, 4915514)
    """
phys_query_2019 = \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    O.CUSTOMER_KEY,
    D.MONTH_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.YR = 2019
    AND
    O.ORDER_PRODUCT_ID IN (4915522, 4915514)
    """

phys_query_2018 = \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    O.CUSTOMER_KEY,
    D.MONTH_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.YR = 2018
    AND
    O.ORDER_PRODUCT_ID IN (4915522, 4915514)
    """

In [124]:
PHYS_2020 = pd.read_sql(con=AMADM, sql=phys_query_2020)
PHYS_2020.head()

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR
0,2020-02-03,-1,52184,2
1,2020-02-24,-1,132928,2
2,2020-02-25,-1,61342,2
3,2020-03-20,-1,56853,3
4,2020-03-30,-1,56691,3


In [134]:
PHYS_2020.groupby('MONTH_NBR').count()

,FULL_DT,ME,CUSTOMER_KEY
MONTH_NBR,,,
1,11643,11643,11643
2,10202,10202,10202
3,10510,10510,10510
4,8956,8956,8956
5,9120,9120,9120
6,8578,8578,8578


In [17]:
PHYS_2019 = pd.read_sql(con=AMADM, sql=phys_query_2019)
PHYS_2019.head()

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR
0,2019-01-25,-1,60515,1
1,2019-01-25,-1,64936,1
2,2019-02-22,-1,56023,2
3,2019-02-26,-1,54083,2
4,2019-03-06,-1,61718,3


In [21]:
PHYS_2018 = pd.read_sql(con=AMADM, sql=phys_query_2018)
PHYS_2018.head()

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR
0,2018-01-23,-1,5499,1
1,2018-01-30,-1,178,1
2,2018-02-19,-1,667,2
3,2018-03-08,-1,667,3
4,2018-03-13,-1,37699,3


In [10]:
cust_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c 
    """

In [11]:
CUST = pd.read_sql(con=AMADM, sql=hosp_cust_query)
CUST.CUSTOMER_KEY = CUST.CUSTOMER_KEY.astype(str)
CUST.head()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,2,000000545697,None,Abdul Haji,Non-Physician Individual,Nonphy,None
2,3,000000552782,None,Mary Tan,Non-Physician Individual,Nonphy,None
3,4,000000647381,None,Osman Kozak,Physician (On Aims),Phys,None
4,5,000000648448,None,Rana Kayal,Non-Physician Individual,Nonphy,None


In [23]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20200620.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [125]:
npi = pd.read_csv('../../Data/Credentialling/Reference_Match.csv')

In [176]:
all_orders = pd.concat([PHYS_2018, PHYS_2019, PHYS_2020])

In [178]:
all_orders = all_orders.sort_values('FULL_DT').drop_duplicates(['ME','CUSTOMER_KEY'], keep='last')

In [179]:
len(all_orders)

270488

In [33]:
ppd = ppd[['ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'FROM_DT', 
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP']]

In [34]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
ppd['ME'] = fix_me(list(ppd['ME']))

In [180]:
all_reapp = pd.merge(all_orders, ppd, on='ME')

In [129]:
len(all_orders)

180801

In [181]:
all_reapp = pd.merge(all_reapp, CUST, on='CUSTOMER_KEY')

In [183]:
all_reapp = all_reapp.fillna('None')
polo_potential = all_reapp[all_reapp['POLO_MAILING_LINE_1']!='None']

In [133]:
all_reapp.columns

Index(['FULL_DT', 'ME', 'CUSTOMER_KEY', 'MONTH_NBR', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME', 'SUFFIX', 'FROM_DT', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE', 'POLO_ZIP',
       'CUSTOMER_NBR', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC'],
      dtype='object')

In [228]:
len(polo_potential)

71025

In [230]:
polo_potential['CUSTOMER']= [x.upper() for x in list(polo_potential['CUSTOMER_NAME'])]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [234]:
count = 0
mes = []
for row in polo_potential.itertuples():
    if fuzz.ratio(row.CUSTOMER, row.POLO_MAILING_LINE_1)>70:
        print(f'{row.CUSTOMER} matches {row.POLO_MAILING_LINE_1}')
        count += 1
        mes.append(row.ME)

CHILDREN'S HOSPITAL COLORADO matches THE CHILDREN'S HOSPITAL
CHILDREN'S HOSPITAL COLORADO matches CHILDREN'S HOSPITAL COLORADO
CHILDREN'S HOSPITAL COLORADO matches THE CHILDRENS HOSPITAL
CHILDREN'S HOSPITAL COLORADO matches THE CHILDREN'S HOSPITAL
LANDMARK MEDICAL CENTER matches WOONSOCKET MEDICAL CENTER
LANDMARK MEDICAL CENTER matches LANDMARK MEDICAL CENTER
LANDMARK MEDICAL CENTER matches LANDMARK MEDICAL CENTER
LANDMARK MEDICAL CENTER matches LANDMARK MEDICAL CENTER
CONE HEALTH matches CONE HEALTH
CONE HEALTH matches NOVANT HEALTH
CONE HEALTH matches CONE HEALTH
CONE HEALTH matches CONE HEALTH
CONE HEALTH matches CONE HEALTH
CONE HEALTH matches CONE HEALTH SYSTEM
HEYWOOD HOSPITAL matches HEYWOOD HOSPITAL
RIVERVIEW REGIONAL MEDICAL CENTER matches WESTERN ARIZONA REGIONAL MEDICAL CENTER
JACKSON HOSPITAL matches JACKSON HOSP
ST. MARY'S REGIONAL MEDICAL CENTER matches ST MARY'S REGIONAL MEDICAL CENTER ED
KINGMAN REGIONAL MEDICAL CENTER matches KINGMAN REGIONAL MEDICAL CENTER
KINGMAN REG

DECATUR COUNTY MEMORIAL HOSPITAL matches DECATUR COUNTY MEMORIAL HOSPITAL
DECATUR COUNTY MEMORIAL HOSPITAL matches DECATUR COUNTY MEMORIAL HOSPITAL
DECATUR COUNTY MEMORIAL HOSPITAL matches DECATUR COUNTY MEMORIAL HOSPITAL
DECATUR COUNTY MEMORIAL HOSPITAL matches DECATUR CNTY MEM HOSP
BELOIT MEMORIAL HOSPITAL matches BELOIT MEMORIAL HOSPITAL
BELOIT MEMORIAL HOSPITAL matches BELOIT MEMORIAL HOSPITAL
BELOIT MEMORIAL HOSPITAL matches BELOIT MEMORIAL HOSPITAL
BELOIT MEMORIAL HOSPITAL matches BELOIT MEMORIAL HOSPITAL
BELOIT MEMORIAL HOSPITAL matches FHN-FREEPORT MEMORIAL HOSPITAL
BELOIT MEMORIAL HOSPITAL matches BELOIT MEMORIAL HOSPITAL
MARGARET MARY HEALTH matches MARY MARGARET HEALTH
MARGARET MARY HEALTH matches MARGARET MARY HEALTH
SOUTH GEORGIA MEDICAL CENTER matches SOUTH GEORGIA MEDICAL CENTER
SOUTH GEORGIA MEDICAL CENTER matches SOUTH GEORGIA MEDICAL CENTER
SOUTH GEORGIA MEDICAL CENTER matches COLQUITT REGIONAL MEDICAL CENTER
SOUTH GEORGIA MEDICAL CENTER matches SOUTH GEORGIA MEDICAL 

SINGING RIVER HEALTH SYSTEM matches SINGING RIVER HEALTH SYSTEM
NORTH COUNTRY HOSPITAL matches NORTH COUNTRY HOSPITAL
VALLEY REGIONAL HOSPITAL matches VALLEY REG HOSP
ST. FRANCIS MEDICAL CENTER matches SANFORD MEDICAL CENTER
ST. FRANCIS MEDICAL CENTER matches OLMSTED MEDICAL CENTER
ST. FRANCIS MEDICAL CENTER matches ST FRANCIS MED CTR
HANCOCK MEDICAL CENTER matches OCHSNER MEDICAL CENTER
HANCOCK MEDICAL CENTER matches HANCOCK MEDICAL CENTER
HANCOCK MEDICAL CENTER matches OCHSNER MEDICAL CENTER
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC P A
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLNC PA
HATTIESBURG CLINIC P.A. matches HATTIESBURG CLINIC DEPT OF PULMONARY
H

SHASTA REGIONAL MEDICAL CENTER matches SHARON REGIONAL MEDICAL CENTER
SHASTA REGIONAL MEDICAL CENTER matches SIDNEY REGIONAL MEDICAL CENTER
OKLAHOMA STATE UNIVERSITY MEDICAL CENTER matches OKLAHOMA STATE UNIVERSITY
SALEM TOWNSHIP HOSPITAL matches SALEM TOWNSHIP HOSPITAL
SALEM TOWNSHIP HOSPITAL matches SALEM TOWNSHIP HOSP ED
PANA COMMUNITY HOSPITAL matches PANA COMMUNITY HOSPITAL
DALLAS MEDICAL CENTER matches DALLAS REGIONAL MEDICAL CT
DALLAS MEDICAL CENTER matches METHODIST DALLAS MEDICAL CENTER
DALLAS MEDICAL CENTER matches DALLAS MED CENTER PLAZA 3
DALLAS MEDICAL CENTER matches DALLAS REGIONAL MEDICAL CENTER
TEXAS SCOTTISH RITE HOSPITAL matches TX SCOTTISH RITE HOSP
TEXAS SCOTTISH RITE HOSPITAL matches TEXAS SCOTTISH RITE HOSPITAL
TEXAS SCOTTISH RITE HOSPITAL matches TEXAS SCOTTISH RITE HOSP FOR CHILDREN
TEXAS SCOTTISH RITE HOSPITAL matches TEXAS SCOTTISH RITE HOSPTIAL
TEXAS SCOTTISH RITE HOSPITAL matches TEXAS SCOTTISH RITE HOSPITAL
TEXAS SCOTTISH RITE HOSPITAL matches TX SCOTTISH R

LONGVIEW REGIONAL MEDICAL CENTER matches LONGVIEW REGIONAL MEDICAL CENTER EMERGEN
LONGVIEW REGIONAL MEDICAL CENTER matches LONGVIEW REGIONAL MEDICAL CENTER
LONGVIEW REGIONAL MEDICAL CENTER matches LONGVIEW REGIONAL MEDICAL CENTER
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDRENS RESEARCH HOSP
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDREN'S HOSPITAL
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDREN'S RESEARCH HOSPITAL
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDES RESEARCH HOSPITAL
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDREN'S RESEARCH HOSPITAL
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDREN'S RESEARCH HOSPITAL
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHLDRNS RESRCH HOS
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDREN'S RESEARCH HOSP
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILDRENS RESEARCH HOSP MS 735
ST. JUDE CHILDREN'S RESEARCH HOSPITAL matches ST JUDE CHILD

...

UINTAH BASIN MEDICAL CENTER matches UINTAH BASIN MEDICAL CENTE
IBERIA MEDICAL CENTER matches IBERIA MEDICAL CENTER
KINGMAN REGIONAL MEDICAL CENTER matches KIMGMAN REGIONAL MEDICAL CENTER
ALLEGHANY MEMORIAL HOSPITAL matches ALLEGHANY MEMORIAL HOSPITAL
CEDAR VALLEY MEDICAL SPECIALISTS, P.C. matches CEDAR VALLEY MEDICAL SPECIALISTS PC
CEDAR VALLEY MEDICAL SPECIALISTS, P.C. matches CEDAR VALLEY MED SPECIALIS
CEDAR VALLEY MEDICAL SPECIALISTS, P.C. matches CEDAR VALLEY MED SPECIALISTS PC
CEDAR VALLEY MEDICAL SPECIALISTS, P.C. matches CEDAR VALLEY MEDICAL SPECIALISTS P C
CEDAR VALLEY MEDICAL SPECIALISTS, P.C. matches CEDAR VALLEY MEDICAL SPECIALISTS PC
ESSEX COUNTY HOSPITAL CENTER matches ESSEX COUNTY HOSPITAL CENTER
PREMISE HEALTH matches PREMISE HEALTH
DOROTHEA DIX PSYCHIATRIC CENTER matches DOROTHEA DIX PSYCHIATRIC CENTE
DOROTHEA DIX PSYCHIATRIC CENTER matches DOROTHEA DIX PSYCHIATRIC CENTER
LEBAUER HEALTHCARE matches LEBAUER HEALTHCARE
LEBAUER HEALTHCARE matches LEBAUER HEALTHCARE
COMPREH

JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
JEFFERSON RADIOLOGY matches JEFERSON RADIOLOGY STE 200
JEFFERSON RADIOLOGY matches JEFFERSON RADIOLOGY PC
FOX VALLEY ORTHOPAEDIC INSTITUTE matches FOX VALLEY ORTHOPAEDIC INSTITUTE
FOX VALLEY ORTHOPAEDIC INSTITUTE matches FOX VALLEY ORTHOPAEDIC INSTITUTE
SOUTHERN REGIONAL AHEC matches SOUTHERN REGIONAL AHEC
THE BUFFALO SURGERY CENTER LLC matches BUFFALO SURGERY CENTER
BROUGHTON HOSPITAL matches BROUGHTON HOSPITAL
BROUGHTON HOSPITAL matches BROUGHTON HOSP
BROUGHTON HOSPITAL matches BROUGHTON HOSP
BROUGHTON HOSPITAL matches BROUGHTON HOSP
BROUGHTON HOSPITAL matches BROUGHTON HOSPITAL
BROUGHTON HOSPITAL matches BROUGHTON HOSPITAL
BROUGHTON HOSPITAL matches BROUGHTON HOSPITAL
BROUGHTON HOSPITAL matches BROUGHTON HOSPITAL
CUMBERLAND HALL HOS

THE CORVALLIS CLINIC SURGERY CENTER matches THE CORVALLIS CLINIC
TRI-COUNTY MENTAL HEALTH SERVICES, INC. matches TRI COUNTY MENTAL HEALTH SERVICES
SARAH BUSH LINCOLN HEALTH CENTER matches SARAH BUSH LINCOLN HEALTH CENTER
LOWCOUNTRY ORTHOPAEDIC ASSOCIATES, PA matches DBA LOWCOUNTRY ORTHOPAEDICS
LOWCOUNTRY ORTHOPAEDIC ASSOCIATES, PA matches DBA LOWCOUNTRY ORTHOPAEDICS
UINTAH BASIN MEDICAL CENTER matches UINTAH BASIN MEDICAL CENTER
UINTAH BASIN MEDICAL CENTER matches UNITAH BASIN MED CTR
UINTAH BASIN MEDICAL CENTER matches UINTAH BASIN MEDICAL CENTER
UINTAH BASIN MEDICAL CENTER matches UINTAH BASIN MEDICAL CENTER
HUNTERDON HEALTHCARE PARTNERS matches HUNTERDON HEALTH CARE AT HICKORY RUN
OPTICARE EYE HEALTH CENTER matches OPTICARE EYE HEALTH CENTERS INC
OPTICARE EYE HEALTH CENTER matches OPTICARE EYE HEALTH CENTERS INC
OPTICARE EYE HEALTH CENTER matches OPTICARE EYE HEALTH CENTERS INC
OPTICARE EYE HEALTH CENTER matches OPICARE EYE HEALTH CENTERS INC
PSYCHIATRIC INSTITUTE OF WASHINGTON matc

THE GASTROENTEROLOGY GROUP OF NORTHERN NEW JERSEY, LLC matches THE GASTROENTEROLOGY GROUP OF NORTHERN N
YUMA REHABILITATION HOSPITAL matches YUMA REHABILITATION HOSPITAL
WARREN STATE HOSPITAL matches WARREN STATE HOSPITAL
WARREN STATE HOSPITAL matches WARREN STATE HOSPITAL
COLUMBIA UNIVERSITY HEALTH SERVICE matches COLUMBIA UNIVERSITY HEALTH SERVICE
TANDEM HEALTH SC matches TANDEM HEALTHCARE
MERCED AMBULATORY ENDOSCOPY CENTER matches MERCED AMBULATORY ENDO CTR
FERTILITY CENTERS OF NEW ENGLAND matches FERTILITY CENTERS OF NEW ENGLAND LLC
FERTILITY CENTERS OF NEW ENGLAND matches FERTILITY CENTERS OF NEW ENGLAND
TAMPA FAMILY HEALTH CENTERS, INC. matches TURLEY FAMILY HEALTH CENTER
TAMPA FAMILY HEALTH CENTERS, INC. matches TAMPA FAMILY HEALTH CENTER
TAMPA FAMILY HEALTH CENTERS, INC. matches TAMPA FAMILY HEALTH CENTER
NORTHERN VIRGINIA MENTAL HEALTH INSTITUTE matches NORTHERN VA MENTAL HEALTH INST
CHILDREN'S SPECIALTY CENTER OF NEVADA matches CHILDREN'S SPECIALTY CENTER OF NEVADA
CHILDREN'S

CARROLLTON ORTHOPAEDIC SURGERY CENTER matches CARROLLTON ORTHOPEDIC CLNC
CARROLLTON ORTHOPAEDIC SURGERY CENTER matches CARROLLTON ORTHOPAEDICS
PHYSICIANS GROUP LLC matches PHYSICIANS GROUP LLC
NORTHWEST NASAL SINUS CENTER matches NORTHWEST NASAL SINUS CENTER
NORTHWEST NASAL SINUS CENTER matches NORTHWEST NASAL SINUS CENTER
MACT HEALTH BOARD, INC matches M A C T HEALTH BOAD
CHAPA-DE INDIAN HEALTH PROGRAM, INC. matches CHAPA DE INDIAN HEALTH
NHAN HOA COMPREHENSIVE HEALTH CARE CLINIC matches NHAN HOA COMPREHENSIVE HEALTH
CENTER FOR SPINE INTERVENTION matches CENTER FOR SPINE INTERVENTIONS P C
OUR LADY OF PEACE RESIDENTIAL HOSPICE matches OUR LADY OF PEACE HOME
OCEAN ORTHOPEDIC ASSOCIATES, PA matches OCEAN ORTHOPEDIC ASSOCIATE
GI ASSOCIATES LLC matches GI ASSOCIATES
CHALMERS P. WYLE VA AMBULATORY CARE CENTER matches VA AMBULATORY CARE CENTER
CHILD GUIDANCE RESOURCE CTRS matches CHILD GUIDANCE RESOURCE CENTER
HUMBOLDT STATE UNIVERSITY matches HUMBOLDT STATE UNIVERSITY
BONATI SPINE INSTITUTE

In [119]:
len(polo_potential)

48194

In [232]:
count

4772

In [50]:
len(polo_potential)

48194

In [51]:
polo_potential

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,...,POLO_CITY,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,CUSTOMER
4,2018-01-03,02507781031,1246,1,JOHNSON,ALAN,RICHARD,None,None,ATLANTA MEDICAL CENTER,...,ATLANTA,GA,30312,000002080947,33MG30520,One Call Care Management,Eprofile Webstore Cusomer,Eprofile,Managed Care,ONE CALL CARE MANAGEMENT
6,2018-01-05,00802810491,1246,1,OLSAVSKY,THOMAS,DONALD,None,None,ADVANCE RADIOLOGY STE 220,...,SHELTON,CT,06484,000002080947,33MG30520,One Call Care Management,Eprofile Webstore Cusomer,Eprofile,Managed Care,ONE CALL CARE MANAGEMENT
7,2018-01-05,03506030918,1246,1,WOLFSON,NATASHA,SUSAN,None,None,UNIVERSITY RADIOLOGY GROUP PC,...,E BRUNSWICK,NJ,8816,000002080947,33MG30520,One Call Care Management,Eprofile Webstore Cusomer,Eprofile,Managed Care,ONE CALL CARE MANAGEMENT
11,2018-01-08,03509710618,1246,1,KOLBER,RONALD,B,None,None,5798 CRANBERRY RD A,...,E BRUNSWICK,NJ,8816,000002080947,33MG30520,One Call Care Management,Eprofile Webstore Cusomer,Eprofile,Managed Care,ONE CALL CARE MANAGEMENT
12,2018-01-08,03509802113,1246,1,KELLER,IRWIN,AARON,None,None,UNIVERSITY RADIOLOGY GROUP PC,...,E BRUNSWICK,NJ,8816,000002080947,33MG30520,One Call Care Management,Eprofile Webstore Cusomer,Eprofile,Managed Care,ONE CALL CARE MANAGEMENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180154,2020-06-24,03506801167,134722,6,SWITZER,DONALD,FRANK,None,None,BUFFALO MEDICAL GROUP,...,WILLIAMSVILLE,NY,14221,000000825329,35HP01013,Olean General Hospital,Eprofile Webstore Customer,Eprofile,Hospital,OLEAN GENERAL HOSPITAL
180157,2020-06-24,03519120967,61274,6,NAM,REBEKAH,None,None,None,NYU LANGONE MEDICAL CENTER,...,NEW YORK,NY,10016,000002687497,None,"Irvine Surgical Partners, Llc",Eprofile Webstore Customer,Eprofile,Health Related,"IRVINE SURGICAL PARTNERS, LLC"
180162,2020-06-25,00401770291,54219,6,DELANEY,RICHARD,DEAN,None,None,DELANEY,...,ORMOND BEACH,FL,32174,000002099173,11AM30645,Tomoka Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care,TOMOKA SURGERY CENTER
180166,2020-06-25,04177911815,54219,6,WISELY,DENISE,None,None,None,VOLUSIA ANESTH ASS,...,DAYTONA BEACH,FL,32118,000002099173,11AM30645,Tomoka Surgery Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care,TOMOKA SURGERY CENTER


In [184]:
all_reapp[all_reapp['POLO_MAILING_LINE_2']=='None']

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
4,2018-02-20,03901690188,1739,2,DARROW,BRUCE,ALVIN,None,None,None,None,None,None,None,000002084348,39HP40158,"Eastern Oklahoma Medical Center, Inc.",Eprofile Webstore Cusomer,Eprofile,Hospital
6,2018-05-25,03979890469,1739,5,ALLEN,RICHARD,GREEN,None,None,None,None,None,None,None,000002084348,39HP40158,"Eastern Oklahoma Medical Center, Inc.",Eprofile Webstore Cusomer,Eprofile,Hospital
9,2019-08-14,03901980887,59440,8,MC CLOY-YOUNG,LAURA,BETH,None,None,None,None,None,None,None,000002102593,39HP02108,Perry Memorial Hospital,Eprofile Webstore Customer,Eprofile,Hospital
12,2019-08-14,00106810435,59440,8,PRITCHARD,RONALD,STEPHEN,None,None,None,None,None,None,None,000002102593,39HP02108,Perry Memorial Hospital,Eprofile Webstore Customer,Eprofile,Hospital
21,2019-08-14,00401110160,59440,8,WRIGHT,STEVEN,MATTHEW,None,None,None,None,None,None,None,000002102593,39HP02108,Perry Memorial Hospital,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269741,2020-05-19,30807833185,59649,5,SYED,KHALID,H,None,None,496 WINFIELD STREET,None,EAST LYNN,MA,01904,000002085695,48AM80723,North Tarrant Area Community Health Center,Eprofile Webstore Customer,Eprofile,Group
269755,2020-05-28,03515200478,55382,5,CHAPMAN,ALEXANDRA,None,None,None,None,None,None,None,None,000002079901,48HP06610,Providence Health Center,Eprofile Webstore Customer,Eprofile,Hospital
269777,2020-06-08,04705170598,53869,6,HUDNUT-BEUMLER,JULIA,MARIAN,None,6.23202e+06,None,None,None,None,None,000002080411,07HP10023,Denver Health,Eprofile Webstore Customer,Eprofile,Hospital
269778,2020-06-09,05501010369,134188,6,WEIDMAN,PAUL,DANIEL,None,None,None,None,None,None,None,000003774744,None,Va Premier,Eprofile Webstore Customer,Eprofile,Managed Care


In [192]:
FOUND_LIST = []
count = 0
for row in OLEAN[OLEAN.FULL_DT>=dt.date(2019,1,1)].itertuples():
    print(f'{row.FIRST_NAME} {row.LAST_NAME}')
    found = False
    for provider in PROVIDERS:
        
        if row.LAST_NAME in provider.upper():
            print(provider)
            found = True
    if found:
        count += 1
    FOUND_LIST.append(found)
    print('---')
print(count)

MICHAEL MARINO
Michael Marino, MD
---
ALPEN PATEL
Arun Patel, M.D.
Rajnikant Patel, MD
Aditi Patel, DMD
---
DHEERENDRA PRASAD
Dheerendra Prasad, M.D.
---
RAJAN GULATI
Rajan Gulati, MD
---
JOSEPH ORIE
Joseph Orie, MD
---
ELI FARHI
Eli Farhi, MD
---
ABDUL CHAUDHRY
Abdul Chaudhry, MD
Monir Chaudhry, MD
---
WILLIAM MILLS
William F. Mills, MD
---
FAIEZ ROUMANI
Fayez Roumani, MD
---
BRETT KARLIK
---
ANIL PRADHAN
---
NEAL YOUNG
Karisa Young, NP
Neal Young, MD
---
GLAUCO MARESCA
G. Michael Maresca, MD
---
VIKTOR CORPUZ
Viktor Corpuz, MD
---
MARK THOMPSON
Mark Thompson, MD
---
ROBERT KAYE
Robert Kaye, MD
---
AIJAZ SHAIKH
Aijaz Shaikh, MD
---
AVNIT GOLTEN
---
DAVID GODFREY
---
MUHAMMAD AKHTAR
M. Javed Akhtar, MD
---
PATRICK ODIA
Patrick O'Odia, MD
---
DELILAH STEPHENS
Delilah Stephens, MD
---
ALAN MAUTZ
Alan Mautz, MD
---
VIJAY IYER
Vijay Iyer, MD
---
SELVAKUMAR CHOCKALINGAM
Selvakumar Chockalingam, MD
---
KWAKU AMEXO
---
JOHN WEINHOLD
John R. Weinhold, MD
---
ZAFAR MIRZA
Zafar Mirza, M.D.
---
W

In [185]:
OLEAN = all_reapp[all_reapp['CUSTOMER_NAME']=='Olean General Hospital']

In [196]:
len(OLEAN[OLEAN.FULL_DT>=dt.date(2020,1,1)])

39

In [200]:
len(OLEAN[(OLEAN.POLO_STATE!='NY')&(OLEAN.POLO_MAILING_LINE_2!='None')])

43

In [210]:
OLEAN.groupby('POLO_CITY').count()

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
POLO_CITY,,,,,,,,,,,,,,,,,,,
ALLENTOWN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BANGOR,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BOSTON,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BRADFORD,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
BROCKPORT,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
BRONX,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BROOKLYN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BUFFALO,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
CLEVELAND,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [57]:
PROVIDERS = ["Israr Abbasi, M.D.",
"Seth P. Achey, PA",
"Moses Aderanti, MD",
"Robin C. Aftuck, CNM",
"Michael Aiello, MD",
"M. Javed Akhtar, MD",
"Michel Akl, MD",
'Adil Al-Humadi, MD',
'Mohaned A. Al-Humadi, MD',
'Mohamoud Ali, MD',
'Saif Aljabab, MD',
'Syed A.I. Ally, MD',
'Nisar Alvi, MD',
'Tracy Andrus, PA',
'Alan Aquilina, M.D.'
'Umeer Ashraf, MD',
'Vindhya Athuluru, MD',
'Obioma Atufunwa, PA',
'Banyar Aung, MD',
'Lyudmila Aurora, DO',
'Celinda Austin-Strick, NP',
'Clement Ayanbadejo, MD',
'Kristine E. Bailey, NP',
'Lori Baker, CRNA',
'Robert Baker, III, PA',
'Thomas Ballard, MD',
'Lori Banse-Fay, NP',
'Jeremy Barnett, MD',
'Vaijayantee Belle, M.D.',
'Kurt Benham, MD',
'Michael Benz, MD',
'Kathleen Bethin, MD',
'Ashish Bhatia, MD',
'Shabir Bhayani, MD',
'Jeffrey K. Boon, PA',
'Ruth Brewer, NP',
'David Brubaker, MD',
'Thomas Bryce, MD',
'Heidi Calhoun-Nudd, CRNA',
'David M. Cappuccio, DDS',
'Gary H. Carl, MD',
'Tamara Chaffee, CNM',
'Pradeep Chaganti, MD',
'Puneet Chahal, DPM',
'Brandon Chan, MD',
'Abdul Chaudhry, MD',
'Monir Chaudhry, MD',
'Tahir Chauhdry, DO',
'Selvakumar Chockalingam, MD',
'Zahid Chohan, MD',
'David Collins, NP',
'Lisa Connery, MD',
'Jonathan Coolidge, MD',
'Viktor Corpuz, MD',
'Vincent Cotroneo, MD',
'Matthew R. Cox, MD',
'Leo Cusumano, M.D.',
'Louise Dallaire, MD',
'Andrew Davidson, NP',
'Eddo de Lang, MD',
'Wendy Dennis, PA',
'David DiMarco, M.D.',
'Ashley Disney, PA',
'Ryan Dockery, MD',
'Susan E. Dougherty, CNM',
'Jad Dughayli, MD',
'Hussein Elattar, MD',
'Kathryn Elenchin, CRNA',
'Brian Ewy, DO',
'Kathleen Fanos, DO',
'Eli Farhi, MD',
'Rebecca Farnsworth, CRNA',
'Aamer Farooki, MD',
'Mark Fenzl, MD',
'John Fitzgerald, MD',
'Scott Flick, PA',
'Simon D. Fung-Kee-Fung, MD',
'Sri Nidhi Ganji, MD',
'Benjamin Ge, MD',
'Judit Gellen, MD',
'Daniel J. Gesek, DMD',
'Arindam Ghatak, MD',
'Kirwin Gibbs, MD',
'Robert Gilfert, D.P.M.',
'Sanford Glantz, MD',
'Joseph Gomez, MD, FACC',
'Anupama Gopinath, MD',
'Gerald Gorman, MD',
'Rajan Gulati, MD',
'Robert Haas, MD',
'Ashley Hanchett, PA',
'Kaitlin Hanmer, MD',
'Gregory B. Hare, MD',
'Steven Herrmann, MD',
'Donald Higgs, MD',
'Ahmad Hilal, M.D.',
'Maurice Hinson, MD',
'Christopher Hobler, RSA',
'Corazon Hoffman, CRNA',
'Deborah C. Hughes, NP',
'Ricardo Ilustre, MD',
'Vijay Iyer, MD',
'William Jaremko, M.D.',
'Muhammed Javed, M.D.',
'Alfonso Jimenez, MD',
'Kathryn Jolles, CRNA',
'Tressa Jordan, DPM',
'Jeffrey Jurek, PA',
'Ara Kassarjian, MD',
'Kanik Kathuria, MD',
'Robert Kaye, MD',
'Margaret H. Kent, NP',
'Gurkaramjit Khaira, MD',
'Anwaar Khan, MD',
'Anwar Khan, MD',
'Imtiaz Khokhar, MD',
'Chee Kim, MD',
'Carolyn Kor, MD',
'Ashok Kothari, M.D.',
'Sushama Kotmire Thandla, MD',
'Robert Krall, M.D.',
'Michael Kuettel, M.D.',
'Dinesh Lahoti, M.D.',
'Daniel Lai, M.D.',
'Vikas Lall, MD',
'Joseph Lamb, PA',
'Stephen Laskowski, DO',
'Amita Leonard, NP',
'Glenn Leonard, MD',
'Jerrold Lerman, MD',
'Fred H. Lewis, MD',
'Andrew Lewis, MD',
"O'Dene Lewis, MD",
'Dmitry Lisenenkov, MD',
'Cindy J. Mack, CNM',
'Safiullah Malik, MD',
'Salwat Malik, M.D.',
'Javaid Malik, MD',
'Nadia Malik, MD',
'Christopher T. Mallavarapu, MD',
'G. Michael Maresca, MD',
'Michael Marino, MD',
'Nicole Marker-Zewe, NP',
'Jonathan Marshall, DO',
'Thayaparan Mathanakaran, MD',
'David Mattson, MD',
'Alan Mautz, MD',
'Theresa McClain, DO',
'Megan McGreevy, MD',
'Kaylee McRae, PA',
'Shahnawaz Meer, MD',
'Naga P. Meka, MD',
'Espiridion Mendez, PA',
'John Miller, D.P.M.',
'David Millhouse, DDS',
'William F. Mills, MD',
'Zafar Mirza, M.D.',
'Elizabeth A. Mitchell, NP',
'Kali Moe, NP',
'Joseph Mollura, MD',
'Lori Morrill, PA',
'William Morris, MD',
'Khushboo Movva, MD',
'Ashok Movva, MD',
'Marc Moyer, PA',
'Brinda Muthuswamy, MD',
'Elie Nehme, MD',
'Jill D. Nelson, PA',
'Kathryn Nelson, PA-C',
'Chiemeka Nwokonko, MD',
"Nancy O'Day, CRNA",
"Denise O'Neil, NP",
"Patrick O'Odia, MD",
'Yasser Omar, MD',
'Joseph Orie, MD',
'uanne Osigweh, MD',
'Arun Patel, M.D.',
'Rajnikant Patel, MD',
'Aditi Patel, DMD',
'Patricia Patterson, NP',
'Brad Pecherzewski, NP',
'Erica Petruzzi, CRNA',
'Kishor Phadke, MD',
'Beth Porter, NP',
'Dheerendra Prasad, M.D.',
'Troy Prather, CRNA',
'Rebecca Pratt, MD',
'Elizabeth M. Printup, DNP',
'Louis Prumbs, MD',
'Bhaskar Pusuluri, MD',
'Clifford Pyne, NP',
'Aparna Ramchandran, MD',
'Robert Ramsdell, MD',
'Rachel Raphael, MD, FACS',
'Shanchiya Ravindradas, MD',
'Shafi Raza, MD',
'Alycia Redlinski, DO',
'Mordechai Rehany, MD',
'Christina Roosa, DO',
'Fayez Roumani, MD',
'Joseph Rusnak, MD',
'Munir Salimi, M.D.',
'Pamela Salzmann, DO',
'Leith Sawalha, MD',
'Adelia Sazonov, DO',
'Stephanie E. Schmieg, PA',
'Terence Schumpert, MD',
'Richard D. Scott, M.D.',
'Katie W. Sekulovski, CNM',
'Eva Selker, PA',
'Cara Sellers, CRNA',
'Baiba Senbruna, MD',
'Philip Senger, MD',
'Ki Won Seo, MD',
'Syed Shah, MD',
'Naveed Shahid, MD',
'Aijaz Shaikh, MD',
'Zia Sheikh, MD',
'David S. Shulman, M.D.',
'Saif Siddiqi, MD',
'Shazia Siddiqi, MD',
'Michael Silverberg, MD',
'Catherine Simkovic, CRNA',
'Amandeep Singh, MD',
'Anurag Kishor Singh, MD',
'Karen L. Sleeman, NP',
'Matthew Smith, MD',
'James Smith, MD',
'Neeta Soni, M.D.',
'Mandy M. Spencer, PA',
'Lilleigh Stein, CRNA',
'Delilah Stephens, MD',
'Howard Stoll III, MD',
'Philip Sullivan, MD',
'Donald Switzer, MD',
'Cyrus Tanhaee, MD',
'Rolyn Te, MD',
'Andrew Tedesco, DDS',
'Eric Ten Brock, M.D.',
'Rachael Territo, NP',
'Srinivas P. Thandla, M.D.',
'Joshua Thomas, CRNA',
'Mark Thompson, MD',
'Paul Tinsley, MD',
'James E. Tkacik, PA',
'Ian Tseng, MD',
'Huseyin Tunceroglu, MD',
'Jairo Ulloa, MD',
'Jennifer Ursoy, PA',
'Rajesh Vaid, MD',
'Mary Vaillancourt, NP',
'James VanDeWall, M.D.',
'Pieter VanHorn, PA',
'Darlene Venezia Skaggs, NP',
'Gust P. Verleni, PA',
'Navyugjit Virk, MD',
'Walter H. Warriner, NP',
'Nicholas Watson, PA',
'Stephen Wei, MD',
'John R. Weinhold, MD',
'Mark J. Welch, MD',
'Darryl Wenner, DO',
'Alexandra Wesley, M.D.',
'Kyle A. Wiktor, NP',
'Casey Wild, NP',
'Sebastian Wilk, MD',
'Keith D. Wilkey, MD',
'Rebekah M. Williams, NP',
'Christian Wilson, MD',
'Ian Wilson, MD',
'Gilbert N. Witte, M.D.',
'haddeus S. Wojcik, M.D.',
'Tomasz Woloszyn, MD',
'Michele Wooten, NP',
'Karisa Young, NP',
'Neal Young, MD',
'AnnMarie Zimmermann, MD'
]

In [58]:
len(PROVIDERS)

276

In [63]:
all_reapp[all_reapp['CUSTOMER_NAME']=='Olean General Hospital']

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
8651,2018-01-08,03506010143,178,1,SALERNO,KILIAN,ELIZABETH,None,None,None,None,None,None,None,000000825329,35HP01013,Olean General Hospital,Institution,Instit,Hospital
8652,2018-01-08,03515830374,178,1,DIONNE,PIERRE,EUGENE,None,None,VA OPC,465 N UNION ST,OLEAN,NY,14760,000000825329,35HP01013,Olean General Hospital,Institution,Instit,Hospital
8653,2018-01-08,49553830124,178,1,SAMUEL,SAM,JAYANTH,None,None,None,2666 W STATE ST,OLEAN,NY,14760,000000825329,35HP01013,Olean General Hospital,Institution,Instit,Hospital
8654,2018-02-13,05104031465,178,2,GREEN,DAVID,ADAIR,None,None,None,None,None,None,None,000000825329,35HP01013,Olean General Hospital,Institution,Instit,Hospital
8655,2018-04-10,70401670562,178,4,HILAL,AHMAD,None,None,None,None,None,None,None,None,000000825329,35HP01013,Olean General Hospital,Institution,Instit,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180152,2020-06-24,03506871017,134722,6,RUSNAK,JOSEPH,GERARD,None,None,None,3050 ORCHARD PARK RD,BUFFALO,NY,14224,000000825329,35HP01013,Olean General Hospital,Eprofile Webstore Customer,Eprofile,Hospital
180153,2020-06-24,04109921608,134722,6,WELCH,MARK,JOS,None,None,None,116 INTERSTATE PKWY,BRADFORD,PA,16701,000000825329,35HP01013,Olean General Hospital,Eprofile Webstore Customer,Eprofile,Hospital
180154,2020-06-24,03506801167,134722,6,SWITZER,DONALD,FRANK,None,None,BUFFALO MEDICAL GROUP,295 ESSJAY RD,WILLIAMSVILLE,NY,14221,000000825329,35HP01013,Olean General Hospital,Eprofile Webstore Customer,Eprofile,Hospital
180155,2020-06-24,03509762502,134722,6,LEWIS,FRED,H,None,None,None,535 MAIN ST,OLEAN,NY,14760,000000825329,35HP01013,Olean General Hospital,Eprofile Webstore Customer,Eprofile,Hospital


In [77]:
all_reapp.FULL_DT>dt.date(2019,6,29)

0         False
1         False
2         False
3         False
4         False
          ...  
180175     True
180176     True
180177     True
180178     True
180179     True
Name: FULL_DT, Length: 180180, dtype: bool

In [115]:
OLEAN['FOUND']=FOUND_LIST

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
OLEAN[OLEAN['FOUND']==True].groupby('POLO_CITY').count()

,FULL_DT,ME,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FOUND
POLO_CITY,,,,,,,,,,,,,,,,,,,,
ALLENTOWN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BOSTON,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BRADFORD,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
BROCKPORT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BRONX,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BROOKLYN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BUFFALO,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
CLEVELAND,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
COVINGTON,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [140]:
npi['CUSTOMER']=[x.upper() for x in list(npi['CUSTOMER_NAME'])]

In [212]:
all_reapp['CUSTOMER']=[x.upper() for x in list(all_reapp['CUSTOMER_NAME'])]

In [213]:
ATTEMPTS = pd.merge(all_reapp, npi, on='CUSTOMER')

In [226]:
all_reapp.groupby('ME').count()

,FULL_DT,CUSTOMER_KEY,MONTH_NBR,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,CUSTOMER
ME,,,,,,,,,,,,,,,,,,,,
00102000013,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
00102000072,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
00102000081,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
00102000170,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
00102000196,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96501880041,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
96501890011,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
96501930013,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [215]:
all_reapp.groupby('CUSTOMER_CATEGORY_DESC').count()[['FULL_DT']]

,FULL_DT
CUSTOMER_CATEGORY_DESC,
Ambulatory Care,51141
Consulting,1400
Credit Verification Org,12947
Government,140
Group,11541
Health Related,8142
Hospital,141588
Long Term Care,654
Managed Care,41478


In [155]:
ATTEMPTS.columns

Index(['FULL_DT', 'ME', 'CUSTOMER_KEY', 'MONTH_NBR', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME', 'SUFFIX', 'FROM_DT', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE', 'POLO_ZIP',
       'CUSTOMER_NBR', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME_x',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC',
       'CUSTOMER', 'CUSTOMER_NAME_y', 'NAME', 'SCORE', 'NPI', 'Taxonomies',
       'OTHER_NAME', 'MAILING_ADDRESS_1', 'MAILING_ADDRESS_2', 'MAILING_CITY',
       'MAILING_STATE', 'MAIING_ZIP', 'MAILING_PHONE', 'LOCATION_ADDRESS_1',
       'LOCATION_ADDRESS_2', 'LOCATION_CITY', 'LOCATION_STATE', 'LOCATION_ZIP',
       'LOCATION_PHONE', 'LAST_UPDATE_DATE'],
      dtype='object')

In [235]:
no_address = 0
state_match = 0
city_match = 0
name_match =0
zip_match = 0
add_match = 0

for row in ATTEMPTS.itertuples():
    
    if row.POLO_MAILING_LINE_2 == 'None':
        no_address+= 1
        continue
    if row.POLO_STATE == row.MAILING_STATE or row.POLO_STATE == row.LOCATION_STATE:
        state_match += 1
        mail_score = fuzz.ratio(row.POLO_MAILING_LINE_2, row.MAILING_ADDRESS_1)
        loc_score = fuzz.ratio(row.POLO_MAILING_LINE_2, row.LOCATION_ADDRESS_1)
        if mail_score > 85 or loc_score > 85:
            add_match+= 1
            mes.append(row.ME)
        if row.POLO_CITY == row.MAILING_CITY or row.POLO_CITY == row.LOCATION_CITY:
            city_match += 1
        if row.POLO_ZIP == row.MAIING_ZIP or  row.POLO_ZIP == row.LOCATION_ZIP:
            zip_match += 1
    if fuzz.ratio(row.POLO_MAILING_LINE_1, row.CUSTOMER)>80:
        name_match+= 1

In [219]:
no_address

16296

In [223]:
zip_match

1271

In [173]:
no_address

10114

In [233]:
len(ATTEMPTS)

148682

In [236]:
len(mes)

10949

In [238]:
ppd[ppd['ME'].isin(mes)]

,ME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,FROM_DT,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP
22,00102000251,BOLLING,ROBERT,PEARCE,NaN,NaN,BOLLING PLASTIC SURGERY,1035 TEMPLE AVE N,FAYETTE,AL,35555
79,00102000901,FALDON,SALIH,O,NaN,NaN,UPSON REGIONAL MEDICAL CENTER,801 W GORDON ST,THOMASTON,GA,30286
224,00102010710,ASURU,AGATHA,TAMUMOIYOWU,NaN,NaN,FRANKLIN PRIMARY HEALTH CENTER INC,424 S WILSON AVE,PRICHARD,AL,36610
599,00102031377,WILLIAMS,SUSANNAH,TAYLOR,NaN,NaN,MEMPHIS MENTAL HEALTH INSTITUTE,951 COURT AVE,MEMPHIS,TN,38103
850,00102050606,SCALICI,JENNIFER,MICHELLE,NaN,NaN,MITCHELL CANCER INST,1660 SPRING HILL AVE,MOBILE,AL,36604
...,...,...,...,...,...,...,...,...,...,...,...
1283065,95702910235,ANTIC,TATJANA,NaN,NaN,NaN,MC 6101,5841 S MARYLAND AVE,CHICAGO,IL,60637
1283105,95702940126,SHAHSAVAR,ABDOLAZIM,NaN,NaN,NaN,SHEPPARD PRATT HEALTH SYSTEM,6501 N CHARLES ST,TOWSON,MD,21204
1283478,95708800016,CVEJIN,SNEZANA,ZIVOJINA,NaN,NaN,R J BLACKLEY ADATC,100 W H ST,BUTNER,NC,27509
1283489,95708810011,BABOVIC,SRDAN,NaN,NaN,NaN,OLMSTED MEDICAL CENTER,210 9TH ST SE,ROCHESTER,MN,55904


In [246]:
all_reapp.columns

Index(['FULL_DT', 'ME', 'CUSTOMER_KEY', 'MONTH_NBR', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME', 'SUFFIX', 'FROM_DT', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE', 'POLO_ZIP',
       'CUSTOMER_NBR', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC',
       'CUSTOMER'],
      dtype='object')

In [253]:
all_reapp[(all_reapp.FULL_DT>=dt.date(2020,5,1))&(all_reapp.FULL_DT<=dt.date(2020,5,31))&(all_reapp.ME.isin(mes))].to_excel('../../Data/Credentialling/Reappointment_Validation_May.xlsx')

In [251]:
all_reapp[(all_reapp.ME.isin(mes))&(all_reapp.FULL_DT>=dt.date(2019,1,1))&(all_reapp.FULL_DT<=dt.date(2019,12,31))].groupby('MONTH_NBR').count()[['FULL_DT']]

,FULL_DT
MONTH_NBR,
1,453
2,389
3,365
4,416
5,483
6,414
7,373
8,473
9,473
